In [1]:
import os, sys # add MaaSSim to path (not needed if MaaSSim is already in path)
module_path = os.path.abspath(os.path.join('../..'))
if module_path not in sys.path:
    sys.path.append(module_path)
import MaaSSim

In [2]:
#!pip install MaaSSim
import MaaSSim.utils
import logging
from dotmap import DotMap
import MaaSSim.simulators
from MaaSSim.data_structures import structures as inData

In [3]:
sim = MaaSSim.simulators.simulate(config = 'glance.json')  # run the simulation from a given configuration
sim.runs[0].trips  # access the results
params = MaaSSim.utils.get_config('glance.json')  # load configuration
params.city = "Nootdorp, Netherlands" # modify it
inData = MaaSSim.utils.load_G(inData,params)  # load the graph for a different city
sim_1 = MaaSSim.simulators.simulate(params=params) # run the simulation
params.nP = 5 # change number of travellers
inData = MaaSSim.utils.prep_supply_and_demand(inData, params)  # regenerate supply and demand
sim_2 = MaaSSim.simulators.simulate(inData=inData,params=params) # run the second simulation
print('Total waiting time: {}s in first simulation and {}s in the second.'.format(sim_1.res[0].pax_exp['WAIT'].sum(),
      sim_2.res[0].pax_exp['WAIT'].sum()))  # compare some results
space =  dict(nP=[5,10,20], nV = [5,10]) # define search space of supply and demand levels
MaaSSim.simulators.simulate_parallel(inData=inData, params=params, search_space = space, logger_level = logging.WARNING) # run parallel experiments
res = MaaSSim.utils.collect_results(params.paths.dumps) # collect results from  parallel experiments

def my_function(**kwargs): # user defined function to represent agent decisions
    veh = kwargs.get('veh', None)  # input from simulation
    sim = veh.sim  # access to simulation object
    if len(sim.runs) > 0:
        if sim.res[last_run].veh_exp.loc[veh.id].nRIDES > 3:
            return False # if I had more than 3 rides yesterday I stay
        else:
            return True # otherwise I leave
    else:
        return True # I do not leave on first day
        
sim = MaaSSim.simulators.simulate(inData=inData,params=params, f_driver_out = my_function, logger_level = logging.INFO) # simulate with my user defined function

16-10-20 14:09:09-WARNING-Setting up 4h simulation at 2020-10-16 12:40:43 for 2 vehicles and 5 passengers in Nootdorp, Netherlands
16-10-20 14:09:09-INFO--------------------	Starting simulation	-------------------
16-10-20 14:09:09-INFO-pax    0  got to dest at                           2020-10-16 12:44:10
16-10-20 14:09:09-INFO-pax    1  got to dest at                           2020-10-16 12:51:19
16-10-20 14:09:09-INFO-pax    2  got to dest at                           2020-10-16 13:11:25
16-10-20 14:09:09-INFO-pax    3  got to dest at                           2020-10-16 14:20:31
16-10-20 14:09:09-INFO-pax    4  got to dest at                           2020-10-16 16:10:51
16-10-20 14:09:09-INFO-veh    1  quitted shift                            2020-10-16 18:40:42
16-10-20 14:09:09-INFO-veh    2  quitted shift                            2020-10-16 18:40:42
16-10-20 14:09:09-INFO--------------------	Simulation over		-------------------
16-10-20 14:09:09-INFO-queue of requests 0
16-10